## Imports

In [55]:
from premise import *
import bw2data as bd
import bw2io as bo
import numpy as np
import pandas as pd
#import brightway2 as bw

### Project set up

In [2]:
bd.projects.set_current("MPTEX")

In [3]:
bo.bw2setup()

Biosphere database already present!!! No setup is needed


In [4]:
bd.databases

Databases dictionary with 3 object(s):
	SSP2-RCP19-2050
	biosphere3
	ecoinvent 3.9.1 cutoff

In [5]:
# file path to the place with the ecoinvent spold files (datasets directory)
fpei39 = "/etc/data/ecospold/datasets"
if 'ecoinvent 3.9.1 cutoff' in bd.databases:
    print("Database has already been imported")
else:
    ei39 = bo.SingleOutputEcospold2Importer(fpei39, 'ecoinvent 3.9.1 cutoff')
    ei39.apply_strategies()
    ei39.statistics()
    
    if len(list(ei39.unlinked)) == 0:
        ei39.write_database()

Database has already been imported


In [116]:
ndb = NewDatabase(
    scenarios=[
        #{"model":"image", "pathway":"SSP2-RCP19", "year":2020},
        #{"model":"image", "pathway":"SSP2-RCP19", "year":2030},
        #{"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        {"model":"remind","pathway":"SSP2-Base","year":2039}
    ],
    source_db="ecoinvent 3.9.1 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=' # <-- decryption key
    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
)

premise v.(2, 1, 0)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [117]:
ndb.update()

Processing scenarios: 100%|█████████████| 1/1 [01:07<00:00, 67.65s/it]

Done!



In [18]:
#Save as a database
ndb.write_db_to_brightway(name=["rmnd-SSP2-Base-2039",])

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Title: Writing activities to SQLite3 database:
  Started: 05/22/2024 18:44:32
  Finished: 05/22/2024 18:45:03
  Total time elapsed: 00:00:30
  CPU %: 99.90
  Memory %: 3.41
Created database: SSP2-RCP19-2050
Generate scenario report.
Report saved under /home/jupyter-xtdds_s2024/MPTEX/export/scenario_report.
Generate change report.
Report saved under /home/jupyter-xtdds_s2024/MPTEX.


In [21]:
# Trouble shooting : 
#ds = bd.get_activity(('biosphere3', 'd3260d0e-8203-4cbb-a45a-6a13131a5108'))
#print(ds['name'])

NMVOC, non-methane volatile organic compounds


## PV dataset look-up

In [23]:
import wurst
database = wurst.extract_brightway2_databases("SSP2-RCP19-2050")

Getting activity data


100%|██████████| 40657/40657 [00:00<00:00, 289588.27it/s]


Adding exchange data to activities


100%|██████████| 1524505/1524505 [00:28<00:00, 52748.02it/s] 


Filling out exchange data


100%|██████████| 40657/40657 [00:01<00:00, 21473.10it/s]


In [24]:
import pickle
with open(r"ei391.pickle", "wb") as output_file:
    pickle.dump(database, output_file)

In [25]:
def load_db():
    with open(r"ei391.pickle", "rb") as input_file:
        db = pickle.load(input_file)

    for ds in db:
        if "categories" in ds:
            del ds["categories"]
            
    return db

In [26]:
db = load_db()

In [57]:
import wurst.searching as ws

e_mixes = ws.get_many(db,
        ws.contains("name","market for electricity")
                     )


#solar_pv = ws.get_many(db, 
#    ws.contains("name", "photovoltaic")
#)

#print(e_mixes)
counter = 0
for ds in e_mixes:
    counter+=1
    #print(ds.keys())
    for exc in ds['exchanges']:
        if 'photovoltaic' in exc['name'] :
            print(ds['name'],ds['location'],'--->',exc['name'])
        #else :
        #    print(ds['name'],exc['name'])

print(counter)
#for ds in e_mixes :
#    print(ds['name'],ds['location'])





market for electricity, low voltage, renewable energy products CH ---> electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted, renewable energy produc
market for electricity, low voltage, renewable energy products CH ---> electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted, renewable energy product
615


In [ ]:
for ds in solar_pv:
    print(ds["name"],ds['location'])

In [112]:
def pv_market_mixes_identifier(db_name) :
    """
    Function to list and parse pv datasets affecting an energy grid mix.
    Market group datasets 
    """
    market_group_ds_storage = []
    market_ds_storage = []
    storage = []

    market_group_ds = []
    market_ds = []

    # Strategy filtering : is the dataset part of a market group or a basic market electricity grid mix 
    for ds in bd.Database(db_name) : 
        if 'market for electricity' in ds['name'] :
            group_counter = 0
            for exc in ds.exchanges(): 
                if 'market group for electricity' in exc['name'] : 
                    group_counter +=1 
            if group_counter == 0 : 
                market_ds.append((ds['name'],ds['code'])) 
            else : 
                market_group_ds.append((ds['name'],ds['location'],exc['input']))

    # Find if there are pv technologies involved with any of the market datasets : 
    for market in market_ds : 
        ds = bd.get_activity((db_name,market[1]))
        for exc in ds.exchanges() :
            if 'photovoltaic' in exc.input['name'] :
                # Extract the dataset name
                dataset_name = exc.input['name'] 
                # Make the name usable
                parsed_name = pv_name_parser(dataset_name)
                # skip non pv energy producing datasets 
                if not any('kWp' in s for s in parsed_name) : 
                    continue  
                # Clear the kWp unit from the name
                parsed_name = [sub.replace('kWp','') for sub in parsed_name]
                market_ds_storage.append((ds['name'],ds['location'],np.nan,*parsed_name))
                
    # Find if there are pv technologies involved with any of the market GROUP datasets : 
    for market_group in market_group_ds : 
        group_ds = bd.get_activity((market_group[2]))
        # Note : the market groups typically have an intermediate dataset splitting commercial/residential
        for exc in group_ds.exchanges(): 
            if 'photovoltaic' in exc.input['name']: 
                production_sector = bd.get_activity((db_name,exc.input['code']))
                # Find the specific pv dataset technology 
                for exc2 in production_sector.exchanges():
                    if 'photovoltaic' in exc2['name'] :
                        # Extract the dataset name
                        dataset_name = exc2['name']
                        # Make the name usable
                        parsed_name = pv_name_parser(dataset_name)
                        # skip non pv energy producing datasets 
                        if not any('kWp' in s for s in parsed_name) : 
                            continue
                        # Clear the kWp unit from the name
                        parsed_name = [sub.replace('kWp','') for sub in parsed_name]
                            
                        market_group_ds_storage.append((market_group[0],market_group[1],group_ds['location'],*parsed_name))

    # Convert to pd dataframe 
    cols = ['Market Dataset','ecoinvent location','Market group location','Installation','Peak power','Technology','Placement','Build','Placement2']

    df1 = pd.DataFrame(market_ds_storage)#,columns = cols))
    df2 = pd.DataFrame(market_group_ds_storage)#,columns = cols)

    return df1,df2


def pv_name_parser(ds_name): 
    """
    Extract technical information from the dataset names
    """
    #print(dataset_name)
    strings = ds_name.replace('electricity production,','')
    strings = strings.replace('photovoltaic','')
    strings = strings.replace('installation','')
    strings = strings.replace('kWp','kWp,')
    strings = str(strings).split(',')

    return strings




the_db = 'ecoinvent 3.9.1 cutoff'
the_other_db = 'SSP2-RCP19-2050'
pv_datasets = pv_market_mixes_identifier(the_other_db)    
display(pv_datasets[0])
display(pv_datasets[1])

,0,1,2,3,4,5,6,7,8,9
0,"market for electricity, low voltage, renewable...",CH,NaN,,3,slanted-roof,single-Si,panel,mounted,renewable energy produc
1,"market for electricity, low voltage, renewable...",CH,NaN,,3,slanted-roof,multi-Si,panel,mounted,renewable energy product


,0,1,2,3,4,5,6,7,8,9
0,"market for electricity, high voltage",KH,SEAS,flat-roof,156,,multi-Si,on roof,None,None
1,"market for electricity, high voltage",KH,SEAS,flat-roof,156,,single-Si,on roof,None,None
2,"market for electricity, high voltage",KH,SEAS,open ground,570,,CIS,on open ground,None,None
3,"market for electricity, high voltage",KH,SEAS,open ground,570,,CdTe,on open ground,None,None
4,"market for electricity, high voltage",KH,SEAS,open ground,570,,micro-Si,on open ground,None,None
...,...,...,...,...,...,...,...,...,...,...
14116,"market for electricity, high voltage",CU,RCAM,open ground,570,,single-Si,on open ground,None,None
14117,"market for electricity, high voltage",CU,RCAM,slanted-roof,93,,multi-Si,laminated,integrated,on roof
14118,"market for electricity, high voltage",CU,RCAM,slanted-roof,93,,multi-Si,panel,mounted,on roof
14119,"market for electricity, high voltage",CU,RCAM,slanted-roof,93,,single-Si,laminated,integrated,on roof


In [240]:
bd.databases

Databases dictionary with 3 object(s):
	SSP2-RCP19-2050
	biosphere3
	ecoinvent 3.9.1 cutoff

In [214]:
storage = []

for ds in bd.Database('SSP2-RCP19-2050') : 
    if 'market for electricity' in ds['name']: 

        for exc1 in ds.exchanges() : 
#            print(exc1)
            if 'market group for electricity' in exc1['name']:
                #print(exc1['name'])
                for exc2 in bd.get_activity(exc1['input']).exchanges():
                #print(bd.get_activity(exc1['input']))
                #for exc2 in exc1.exchanges(): 
                    if 'photovoltaic' in exc2['name']:
                        #print(exc2['name'],bd.get_activity(exc2['input'])['name'])
                        
                        for exc3 in bd.get_activity(exc2['input']).exchanges(): 
                            #print(exc3['name'])
                            if 'photovoltaic' in exc3['name'] : 
                                strings = str(exc3['name']).split(',')
                                #print(strings)

                                # Clear non production datasets
                                if not any('kWp' in s for s in strings) : 
                                    #print('here')
                                    continue

                                # Edit the strings :
                                if 'MWp' in strings[1] : 
                                    strings[1] = 1000*int(strings[1].replace('MWp',''))
                                if 'kWp' in strings[1] : 
                                    strings[1] = int(strings[1].replace('kWp',''))
                                if 'installation' in strings[0] : 
                                    strings[0] = strings[0].replace('installation','')
                                if 'photovoltaic' in strings[0] : 
                                    strings[0] = strings[0].replace('photovoltaic','')

                                storage.append((ds['name'],ds['location'],exc1['location'],*strings))

            
import pandas as pd
df = pd.DataFrame(storage)
df.columns= ['Market Dataset','ecoinvent location','Market group location','Installation','Peak power','Technology','Placement','Build','Placement2']
display(df)

import pickle
with open(r"pv_details.pickle", "wb") as output_file:
    pickle.dump(df, output_file)

,Market Dataset,ecoinvent location,Market group location,Installation,Peak power,Technology,Placement,Build,Placement2
0,"market for electricity, high voltage",US-MRO,USA,flat-roof,156,multi-Si,on roof,None,None
1,"market for electricity, high voltage",US-MRO,USA,flat-roof,156,single-Si,on roof,None,None
2,"market for electricity, high voltage",US-MRO,USA,open ground,570,CIS,on open ground,None,None
3,"market for electricity, high voltage",US-MRO,USA,open ground,570,CdTe,on open ground,None,None
4,"market for electricity, high voltage",US-MRO,USA,open ground,570,micro-Si,on open ground,None,None
...,...,...,...,...,...,...,...,...,...
14116,"market for electricity, high voltage",CN-NWG,CHN,open ground,570,single-Si,on open ground,None,None
14117,"market for electricity, high voltage",CN-NWG,CHN,slanted-roof,93,multi-Si,laminated,integrated,on roof
14118,"market for electricity, high voltage",CN-NWG,CHN,slanted-roof,93,multi-Si,panel,mounted,on roof
14119,"market for electricity, high voltage",CN-NWG,CHN,slanted-roof,93,single-Si,laminated,integrated,on roof


In [100]:
ds = bd.Database('SSP2-RCP19-2050').random()
print(ds.as_dict().keys())
print(ds['name'])
for exc in ds.exchanges() : 
    print(exc.input['name'])
    #print(exc)
#print(ds.exc.upstream())

dict_keys(['location', 'name', 'reference product', 'unit', 'database', 'code', 'comment', 'log parameters'])
hydrogen supply, from gasification of biomass, with CCS, by truck, as liquid, over 500 km
hydrogen supply, from gasification of biomass, with CCS, by truck, as liquid, over 500 km
Hydrogen
hydrogen production, gaseous, 25 bar, from gasification of woody biomass in entrained flow gasifier, with CCS, at gasification plant
hydrogen refuelling station
transport, freight, lorry, unspecified, long haul
market group for electricity, low voltage
transport, freight, lorry, unspecified, long haul
transport, freight, lorry, unspecified, long haul
transport, freight, lorry, unspecified, long haul
transport, freight, lorry, unspecified, long haul


Signature: ds.upstream(kinds=('technosphere',))
Docstring: <no docstring>
Source:   
    def upstream(self, kinds=("technosphere",)):
        return Exchanges(
            self.key,
            kinds=kinds,
            reverse=True
        )
File:      /opt/tljh/user/envs/wurst/lib/python3.11/site-packages/bw2data/backends/peewee/proxies.py
Type:      method

In [81]:
solar_pv_ds = []
for ds in bd.Database('SSP2-RCP19-2050') : 
    if 'kWp' in ds['name']:
        print(ds['name'])

electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted
electricity production, photovoltaic, 3kWp flat-roof installation, single-Si
photovoltaic flat-roof installation, 3kWp, multi-Si, on roof
electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted
photovoltaic open ground installation, 570 kWp, multi-Si, on open ground
electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted
electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted
photovoltaic plant construction, 570kWp, multi-Si, on open ground
electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted
photovoltaic facade installation, 3kWp, multi-Si, laminated, integrated, at building
electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted
electricity production, photovoltaic, 570kWp open ground installatio